# Conditional Logic with If Objects

This notebook demonstrates how to use If objects for conditional logic in pollywog calculations. You'll learn:
- Basic If/otherwise patterns
- Multiple conditions
- Nested conditionals
- Practical mining examples

In [ ]:
import pollywog as pw
from pollywog.core import CalcSet, Number, Category, If
from pollywog.display import display_calcset, set_theme

set_theme("light")

## 1. Basic If/Otherwise Pattern

The simplest conditional: if condition is true, use value A, otherwise use value B.

In [ ]:
# Simple threshold: use grade if above cutoff, otherwise 0
cutoff_calc = Number(
    "Au_cutoff",
    If(
        ("[Au] > 0.5", "[Au]"),
        "0"  # otherwise parameter
    ),
    comment_equation="Apply 0.5 g/t cutoff"
)

calcset_basic = CalcSet([cutoff_calc])
display_calcset(calcset_basic)

## 2. Multiple Conditions

Use multiple condition-value pairs to create multi-way branches.

In [ ]:
# Classify ore based on grade ranges
ore_class = Category(
    "ore_class",
    If(
        [
            ("[Au] < 0.3", "'waste'"),
            ("[Au] < 1.0", "'low_grade'"),
            ("[Au] < 3.0", "'medium_grade'"),
        ],
        "'high_grade'"  # everything else
    ),
    comment_equation="Classify blocks by Au grade"
)

calcset_multi = CalcSet([ore_class])
display_calcset(calcset_multi)

## 3. Complex Conditions

Conditions can use logical operators (and, or, not) and multiple variables.

In [ ]:
# Multi-commodity classification
complex_class = Category(
    "commodity_focus",
    If(
        [
            ("[Au] > 1.0 and [Cu] < 0.2", "'gold_zone'"),
            ("[Cu] > 0.5 and [Au] < 0.3", "'copper_zone'"),
            ("[Au] > 0.5 and [Cu] > 0.3", "'poly_metallic'"),
        ],
        "'waste'"
    ),
    comment_equation="Classify by dominant commodity"
)

# Recoverable metal with conditional adjustments
recoverable = Number(
    "Au_recoverable",
    If(
        [
            ("[rock_type] == 'oxide'", "[Au] * 0.85"),  # 85% recovery in oxide
            ("[rock_type] == 'sulfide'", "[Au] * 0.92"),  # 92% recovery in sulfide
            ("[rock_type] == 'transition'", "[Au] * 0.78"),  # 78% in transition
        ],
        "[Au] * 0.80"  # default 80%
    ),
    comment_equation="Recovery varies by rock type"
)

calcset_complex = CalcSet([complex_class, recoverable])
display_calcset(calcset_complex)

## 4. Nested If Statements

You can nest If objects within the result expressions for decision trees.

In [ ]:
# Nested classification: first by domain, then by grade within domain
nested_class = Category(
    "detailed_class",
    If(
        [
            (
                "[domain] == 'oxide'",
                If(
                    [
                        ("[Au] > 1.0", "'oxide_high'"),
                        ("[Au] > 0.3", "'oxide_low'"),
                    ],
                    "'oxide_waste'"
                )
            ),
            (
                "[domain] == 'sulfide'",
                If(
                    [
                        ("[Au] > 0.8", "'sulfide_high'"),
                        ("[Au] > 0.2", "'sulfide_low'"),
                    ],
                    "'sulfide_waste'"
                )
            ),
        ],
        "'waste'"
    ),
    comment_equation="Domain-specific grade classification"
)

calcset_nested = CalcSet([nested_class])
display_calcset(calcset_nested)

## 5. Practical Mining Example: Mill Feed Decision

Complex real-world example: deciding mill feed strategy based on multiple factors.

In [ ]:
# Determine processing method based on grade and metallurgy
mill_feed = CalcSet([
    # Calculate net smelter return equivalent
    Number(
        "NSR_equiv",
        "[Au] * 50 + [Ag] * 0.8 + [Cu] * 7000",
        comment_equation="USD per tonne, approximate"
    ),
    
    # Determine processing route
    Category(
        "processing_route",
        If(
            [
                # High-grade bypasses crushing, direct to mill
                ("[NSR_equiv] > 100 and [hardness] < 15", "'direct_mill'"),
                
                # Medium grade gets standard processing
                ("[NSR_equiv] > 50", "'standard_crush_mill'"),
                
                # Low grade but high tonnage: heap leach for oxide
                ("[NSR_equiv] > 20 and [rock_type] == 'oxide'", "'heap_leach'"),
                
                # Marginal grade: stockpile for later
                ("[NSR_equiv] > 15", "'stockpile'"),
            ],
            "'waste'"
        ),
        comment_equation="Processing decision tree"
    ),
    
    # Calculate priority score for scheduling
    Number(
        "priority_score",
        If(
            [
                ("[processing_route] == 'direct_mill'", "10"),
                ("[processing_route] == 'standard_crush_mill'", "5"),
                ("[processing_route] == 'heap_leach'", "3"),
                ("[processing_route] == 'stockpile'", "1"),
            ],
            "0"
        ),
        comment_equation="Priority for mine scheduling"
    ),
])

display_calcset(mill_feed)
mill_feed.to_lfcalc("mill_feed_decision.lfcalc")

## 6. Handling Missing Data

Use If statements to handle special values and missing data gracefully.

In [ ]:
# Handle missing/invalid data
data_cleaning = CalcSet([
    # Replace missing values with defaults
    Number(
        "Au_clean",
        If(
            [
                ("not is_normal([Au])", "0.001"),  # Use tiny value for missing
                ("[Au] < 0", "0.001"),  # Replace negatives
            ],
            "[Au]"
        ),
        comment_equation="Clean missing and invalid Au values"
    ),
    
    # Flag for data quality
    Category(
        "data_quality",
        If(
            [
                ("not is_normal([Au]) or not is_normal([density])", "'missing'"),
                ("[Au] < 0 or [density] < 1", "'invalid'"),
                ("[Au] > 100 or [density] > 5", "'outlier'"),
            ],
            "'good'"
        ),
        comment_equation="Flag data quality issues"
    ),
])

display_calcset(data_cleaning)

## Summary

**Key takeaways:**
- Use tuple format: `(condition, result)` for each branch
- Multiple conditions are evaluated in order (first match wins)
- `otherwise` parameter catches all unmatched cases
- Nest If objects for complex decision trees
- Conditions support full Leapfrog expression syntax

**Common patterns:**
- Thresholding: `If((condition, value), default)`
- Classification: Multiple condition-category pairs
- Data cleaning: Handle special values with conditions
- Decision trees: Nest If objects for hierarchical logic